In [ ]:
# For reading credentials from the .env file
import os
from dotenv import load_dotenv
import pandas as pd

from langchain.document_loaders import PyPDFLoader, DataFrameLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# Load API credentials from .env file
load_dotenv()
try:
    API_KEY = os.environ.get("API_KEY")
    project_id = os.environ.get("PROJECT_ID")
except KeyError:
    API_KEY = input("Please enter your WML api key (hit enter): ")
    project_id = input("Please enter your project_id (hit enter): ")

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}


def get_model(model_type, max_tokens, min_tokens, decoding, temperature):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials=credentials,
        project_id=project_id
    )

    return model


def get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature):

    base_model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)
    langchain_model = WatsonxLLM(model=base_model)

    return langchain_model

In [ ]:
import chromadb
import pandas as pd
# Test answering questions based on the provided dataframe
question = "What are the limitations of generative AI models?"
# Provide the path relative to the dir in which the script is running
file_path = "../data/output.pkl"
# 1. Load the dataframe
df = pd.read_pickle(file_path)
df.insert(0, "ID", df.index.astype(str))
chroma_client = chromadb.Client()
collection_name = "my_vector_collection"
# Delete the existing collection 
#chroma_client.delete_collection(collection_name)
try:
    print("We create a Colletion:",collection_name)
    collection = chroma_client.create_collection(
        name=collection_name)  
except:
    print("We load a Colletion:",collection_name)
    collection = chroma_client.get_collection(name=collection_name)
def create_embeddings(text):
    from langchain_community.embeddings import HuggingFaceEmbeddings
    import numpy as np  # Optional, for numerical operations on embeddings (if needed)
    # Choose an appropriate model:
    model_name = "sentence-transformers/all-mpnet-base-v2"  # Replace with your desired model if needed
    # Set device (CPU or GPU) based on your hardware and performance requirements:
    model_kwargs = {'device': 'cpu'}  # Change to 'cuda' for GPU usage (if available)
    # Encoding options (normalization is often recommended):
    encode_kwargs = {'normalize_embeddings': True}  # Experiment with normalization
    # Initialize the embedding model:
    hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)    
    embedding = hf.embed_query(text)  # Use embed_query for single text
    return embedding

def create_embeddings_and_store(df, page_content_column, collection):
    """
    This function generates embeddings from a dataframe and stores them in a Chroma collection.

    Args:
        df (pandas.DataFrame): The dataframe containing text data.
        page_content_column (str): The name of the column containing the text content.
        collection (chromadb.Collection): The Chroma collection to store the embeddings.
    """
    # Extract the text content directly for Chroma's preferred format
    documents = df[page_content_column].tolist()
    for index, row in df.iterrows():
        text = row[page_content_column]
        # Corrected embedding generation:
        embedding = create_embeddings(text)  # Use embed_query for single text
        # Corrected embedding extraction (no tolist() needed):
        embedding_values = embedding  # The embedding is already a list of floats
        # Print each document, embedding pair for debugging or verification (optional)
        print(f"Document: {text}\nEmbedding: {embedding_values}")
    # Insert data into Chroma collection using preferred format
    collection.add(documents=documents, ids=df['ID'].tolist())
# 3. Create embeddings and store them in Chroma (if embeddings don't exist)
# Comment out this step if the embeddings are already created and stored
create_embeddings_and_store(df, "Content", collection)    

In [ ]:
question = 'What are the limitations of generative AI models?'
embedding_question = embedding = create_embeddings(question)

In [ ]:
# Specify model parameters (unchanged)
model_type = "meta-llama/llama-2-70b-chat"
max_tokens = 300
min_tokens = 100
decoding = DecodingMethods.GREEDY
temperature = 0.7
# Get the LangChain model (unchanged)
model = get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature)
context = collection.query(query_texts=question, n_results=2)
prompt_template = '''Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base.  
{context}
Question: {question}
'''
from langchain_core.prompts import PromptTemplate
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}